# GIC Analysis

Calculating Geomagnetically Induced Currents (GIC) for a uniform electric field.

In [ ]:
from esapp import GridWorkBench, GICXFormer

wb = GridWorkBench(r"case.pwb")

'open' took: 2.8327 sec


Calculate GIC for a uniform 1.0 V/km E-field oriented at 90 degrees.

In [ ]:
wb.calculate_gic(max_field=1.0, direction=90.0)

### Retrieve Transformer GIC

In [3]:
gics = wb[GICXFormer, GICXFormer.GICXFNeutralAmps]
gics

,BusNum3W,BusNum3W:1,BusNum3W:2,GICXFNeutralAmps
0,1,2,0,0.753270
1,1,2,0,0.753270
2,1,2,0,0.753270
3,5,6,0,13.443257
4,9,10,0,15.439837
5,14,15,0,12.639866
6,14,15,0,12.639866
7,22,23,0,-0.009016
8,22,23,0,-0.009016
9,22,23,0,-0.009016


And find transformer with max GICs

In [6]:
gics["GICXFNeutralAmps"].max()

np.float64(15.439837455749512)